# Kapitola 25: K-Nearest Neighbors (KNN) - Jsi takovy jako tvoji pratele

## Algoritmus zalozeny na hlasovani sousedu

Predstavte si, ze jste na vecirek a nikho nezante. Chcete odhadnout, kdo je fanouškem sci-fi filmu. Co udelate? Podivate se na lidi kolem vas. Pokud se tri nejblizsi bavi o Star Wars, pravdepodobne jste u fanoušku sci-fi!

Prave jste pouzili algoritmus **K-Nearest Neighbors (KNN)**!

---

### Co se naucime:
1. **Princip KNN** - jak funguje hlasovani sousedu
2. **Euklidovska vzdalenost** - jak merime 'blízkost'
3. **Volba parametru K** - kolik sousedu se ptat
4. **Rozhodovaci hranice** - vizualizace, jak AI 'mysli'
5. **Overfitting vs Underfitting** - K prilis male vs prilis velke

### Proc je KNN dulezity?
- Jeden z **nejjednodussich** ML algoritmu
- **Intuitivni** - snadno pochopitelny
- Funguje dobre pro mnoho problemu
- Zadne 'uceni' - pamatuje si vsechna data

## 1. Instalace a import knihoven

In [ ]:
# Instalace knihoven (pro Google Colab)
!pip install pandas numpy matplotlib seaborn scikit-learn -q

print("Knihovny nainstalovany!")

In [ ]:
# Import knihoven
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris, make_moons, make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap

# Nastaveni vizualizaci
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
np.random.seed(42)

print("Vsechny knihovny uspesne nacteny!")

## 2. Princip KNN: Hlasovani sousedu

KNN funguje ve trech jednoduchych krocich:

1. **Zvol K** - kolik sousedu se ptat (napr. K=3)
2. **Najdi K nejblizsich sousedu** - pomoci vzdalenosti
3. **Hlasovani** - ktera trida je nejcastejsi mezi sousedy?

### Euklidovska vzdalenost

```
vzdalenost = √((x₁-x₂)² + (y₁-y₂)²)
```

Je to proste vzdalenost "vzdusnou carou" mezi dvema body.

In [ ]:
# Demonstrace principu KNN
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Vytvorime jednoducha data
np.random.seed(42)
# Trida A (cervena)
X_A = np.random.randn(15, 2) + np.array([0, 0])
# Trida B (modra)
X_B = np.random.randn(15, 2) + np.array([3, 3])

# Novy bod k klasifikaci
novy_bod = np.array([1.5, 2])

# Graf 1: Data
axes[0].scatter(X_A[:, 0], X_A[:, 1], c='red', s=80, label='Trida A', alpha=0.7)
axes[0].scatter(X_B[:, 0], X_B[:, 1], c='blue', s=80, label='Trida B', alpha=0.7)
axes[0].scatter(novy_bod[0], novy_bod[1], c='green', s=200, marker='*', label='Novy bod (???)')
axes[0].set_title('1. Mame data a novy bod')
axes[0].legend()

# Graf 2: Najdeme K=3 nejblizsi sousedy
axes[1].scatter(X_A[:, 0], X_A[:, 1], c='red', s=80, label='Trida A', alpha=0.7)
axes[1].scatter(X_B[:, 0], X_B[:, 1], c='blue', s=80, label='Trida B', alpha=0.7)
axes[1].scatter(novy_bod[0], novy_bod[1], c='green', s=200, marker='*')

# Vypocteme vzdalenosti a najdeme 3 nejblizsi
X_all = np.vstack([X_A, X_B])
y_all = np.array([0]*15 + [1]*15)  # 0=A, 1=B
vzdalenosti = np.sqrt(np.sum((X_all - novy_bod)**2, axis=1))
nejblizsi_idx = np.argsort(vzdalenosti)[:3]

# Nakreslime cary k 3 nejblizsim
for idx in nejblizsi_idx:
    color = 'red' if y_all[idx] == 0 else 'blue'
    axes[1].plot([novy_bod[0], X_all[idx, 0]], [novy_bod[1], X_all[idx, 1]], 
                 c=color, linestyle='--', linewidth=2)
    axes[1].scatter(X_all[idx, 0], X_all[idx, 1], c=color, s=150, edgecolors='black', linewidth=2)

axes[1].add_patch(plt.Circle(novy_bod, vzdalenosti[nejblizsi_idx[-1]]*1.1, 
                              fill=False, color='green', linestyle='--'))
axes[1].set_title('2. Najdeme K=3 nejblizsi sousedy')
axes[1].legend()

# Graf 3: Hlasovani
sousedi_tridy = y_all[nejblizsi_idx]
pocet_A = np.sum(sousedi_tridy == 0)
pocet_B = np.sum(sousedi_tridy == 1)

axes[2].bar(['Trida A', 'Trida B'], [pocet_A, pocet_B], color=['red', 'blue'], alpha=0.7)
axes[2].set_ylabel('Pocet hlasu')
axes[2].set_title('3. Hlasovani: Kdo vyhrava?')

vitez = 'A' if pocet_A > pocet_B else 'B'
axes[2].annotate(f'Vitez: Trida {vitez}!', xy=(0.5, 2.5), fontsize=14, fontweight='bold',
                 color='green', ha='center')

plt.suptitle('Princip KNN: Hlasovani sousedu', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\nNovy bod bude zarazen do TRIDY {vitez}")
print(f"(hlasovani: {pocet_A} pro A, {pocet_B} pro B)")

## 3. Prakticka klasifikace: Iris dataset

**Iris dataset** je slavny dataset obsahujici mereni tri druhu kosatcu:
- Setosa
- Versicolor
- Virginica

Pro jednoduchost pouzijeme jen dve vlastnosti a dva druhy.

In [ ]:
# Nacteni Iris datasetu
iris = load_iris()

# Pouzijeme jen Versicolor a Virginica (indexy 50-150)
# A jen dva priznaky: delka a sirka okvetniho listku
X = iris.data[50:, 2:]  # Posledni dva sloupce
y = iris.target[50:]

# Prevedeme labely na 0 a 1
y = y - 1  # 1,2 -> 0,1

print("=" * 50)
print("IRIS DATASET")
print("=" * 50)
print(f"\nPocet vzorku: {len(X)}")
print(f"Priznaky: delka a sirka okvetniho listku (cm)")
print(f"Tridy: 0=Versicolor, 1=Virginica")

# Vizualizace
plt.figure(figsize=(10, 6))
plt.scatter(X[y==0, 0], X[y==0, 1], c='red', s=80, label='Versicolor', alpha=0.7)
plt.scatter(X[y==1, 0], X[y==1, 1], c='blue', s=80, label='Virginica', alpha=0.7)
plt.xlabel('Delka okvetniho listku (cm)')
plt.ylabel('Sirka okvetniho listku (cm)')
plt.title('Iris Dataset - Dva druhy kosacu')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Trenink KNN modelu
print("=" * 50)
print("TRENINK KNN MODELU")
print("=" * 50)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"\nTrenovaci data: {len(X_train)} vzorku")
print(f"Testovaci data: {len(X_test)} vzorku")

# Vytvoreni a trenink modelu s K=5
k = 5
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

print(f"\nModel KNN s K={k} natrenovany!")

# Predikce
y_pred = knn.predict(X_test)

# Vyhodnoceni
accuracy = accuracy_score(y_test, y_pred)
print(f"\nPresnost na testovacich datech: {accuracy*100:.1f}%")

## 4. Vizualizace rozhodovaci hranice

**Rozhodovaci hranice** ukazuje, kde v prostoru AI "meni nazor" - kde konci uzemi jedne tridy a zacina uzemi druhe.

In [ ]:
# Funkce pro vizualizaci rozhodovaci hranice
def vizualizuj_hranici(X, y, k, ax=None, title=None):
    """Vizualizuje rozhodovaci hranici KNN modelu"""
    
    # Trenink modelu
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X, y)
    
    # Barevna mapa
    cmap_light = ListedColormap(['#FFAAAA', '#AAAAFF'])
    cmap_bold = ListedColormap(['#FF0000', '#0000FF'])
    
    # Vytvoreni mrizky
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))
    
    # Predikce pro kazdý bod mrizky
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Vykresleni
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))
    
    ax.pcolormesh(xx, yy, Z, cmap=cmap_light, shading='auto')
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold, edgecolor='k', s=50)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    
    if title:
        ax.set_title(title)
    else:
        ax.set_title(f'Rozhodovaci hranice KNN (K={k})')
    
    ax.set_xlabel('Delka okvetniho listku (cm)')
    ax.set_ylabel('Sirka okvetniho listku (cm)')
    
    return model

# Vizualizace pro K=5
plt.figure(figsize=(10, 6))
vizualizuj_hranici(X, y, k=5)
plt.show()

print("Barevne pozadi ukazuje, jak model rozhoduje v kazdem bode prostoru.")

## 5. Vliv parametru K: Overfitting vs Underfitting

**K=1 (prilis male):** Model je velmi citlivy na jednotlive body = **OVERFITTING**

**K=velke:** Model je prilis zjednodusujici = **UNDERFITTING**

In [ ]:
# Porovnani ruznych hodnot K
hodnoty_k = [1, 3, 5, 15, 30]

fig, axes = plt.subplots(1, len(hodnoty_k), figsize=(20, 4))

for ax, k in zip(axes, hodnoty_k):
    vizualizuj_hranici(X, y, k, ax=ax, title=f'K = {k}')

plt.suptitle('Vliv hodnoty K na rozhodovaci hranici', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nK=1: Hranice je 'zubata' - model reaguje na kazdy bod (OVERFITTING)")
print("K=15: Hranice je hladka - model zobecnuje")
print("K=30: Hranice je prilis jednoducha - model ignoruje lokalni struktury (UNDERFITTING)")

In [ ]:
# Analyza vlivu K na presnost
print("=" * 50)
print("ANALYZA VLIVU K NA PRESNOST")
print("=" * 50)

hodnoty_k = range(1, 31)
train_scores = []
test_scores = []

for k in hodnoty_k:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    train_acc = accuracy_score(y_train, knn.predict(X_train))
    test_acc = accuracy_score(y_test, knn.predict(X_test))
    
    train_scores.append(train_acc)
    test_scores.append(test_acc)

# Najdeme nejlepsi K
nejlepsi_k = hodnoty_k[np.argmax(test_scores)]
nejlepsi_score = max(test_scores)

print(f"\nNejlepsi K: {nejlepsi_k}")
print(f"Nejlepsi test presnost: {nejlepsi_score*100:.1f}%")

In [ ]:
# Vizualizace Train vs Test skore
plt.figure(figsize=(12, 5))

plt.plot(hodnoty_k, train_scores, 'b-o', linewidth=2, markersize=4, label='Train presnost')
plt.plot(hodnoty_k, test_scores, 'r-s', linewidth=2, markersize=4, label='Test presnost')
plt.fill_between(hodnoty_k, train_scores, test_scores, alpha=0.2, color='purple')

plt.axvline(x=nejlepsi_k, color='green', linestyle='--', linewidth=2, 
            label=f'Nejlepsi K={nejlepsi_k}')

plt.xlabel('Hodnota K (pocet sousedu)')
plt.ylabel('Presnost')
plt.title('Vliv hodnoty K na presnost modelu')
plt.legend()
plt.grid(True, alpha=0.3)

# Anotace oblasti
plt.annotate('OVERFITTING\n(K prilis male)', xy=(2, 0.85), fontsize=10, color='red')
plt.annotate('UNDERFITTING\n(K prilis velke)', xy=(25, 0.85), fontsize=10, color='orange')

plt.tight_layout()
plt.show()

## 6. Hledani optimalniho K pomoci Cross-Validation

In [ ]:
# Cross-validation pro nalezeni nejlepsiho K
print("=" * 50)
print("CROSS-VALIDATION PRO HLEDANI K")
print("=" * 50)

cv_scores = []
cv_stds = []

for k in hodnoty_k:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X, y, cv=5)
    cv_scores.append(scores.mean())
    cv_stds.append(scores.std())

# Najdeme nejlepsi K
nejlepsi_k_cv = hodnoty_k[np.argmax(cv_scores)]
nejlepsi_cv = max(cv_scores)

print(f"\nNejlepsi K (podle CV): {nejlepsi_k_cv}")
print(f"Prumerne CV skore: {nejlepsi_cv*100:.1f}%")

# Vizualizace
plt.figure(figsize=(12, 5))
plt.errorbar(hodnoty_k, cv_scores, yerr=cv_stds, fmt='-o', capsize=3, 
             color='blue', label='CV skore +/- std')
plt.axvline(x=nejlepsi_k_cv, color='green', linestyle='--', linewidth=2,
            label=f'Nejlepsi K={nejlepsi_k_cv}')
plt.xlabel('Hodnota K')
plt.ylabel('Prumerne CV skore')
plt.title('5-Fold Cross-Validation pro vyber K')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Slozitejsi priklad: Make Moons dataset

In [ ]:
# Make Moons - slozitejsi tvar dat
X_moons, y_moons = make_moons(n_samples=200, noise=0.25, random_state=42)

print("=" * 50)
print("MAKE MOONS DATASET")
print("=" * 50)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Horni rada: Data a rozne K
axes[0, 0].scatter(X_moons[y_moons==0, 0], X_moons[y_moons==0, 1], c='red', s=50, alpha=0.7)
axes[0, 0].scatter(X_moons[y_moons==1, 0], X_moons[y_moons==1, 1], c='blue', s=50, alpha=0.7)
axes[0, 0].set_title('Data: Make Moons')

# Ruzne K
for ax, k in zip([axes[0,1], axes[0,2], axes[1,0], axes[1,1], axes[1,2]], [1, 3, 7, 15, 30]):
    vizualizuj_hranici(X_moons, y_moons, k, ax=ax, title=f'K = {k}')

plt.suptitle('KNN na Make Moons datasetu - vliv K', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nMake Moons ma slozitejsi tvar - linearni metody by selhaly!")
print("KNN dokaze zachytit nelinearni hranice mezi tridami.")

## 8. Shruti a kontrolni otazky

### Co jsme se naucili:

| Koncept | Popis |
|---------|-------|
| **KNN** | Klasifikuje podle hlasovani K nejblizsich sousedu |
| **Parametr K** | Pocet sousedu, ktere se ptame |
| **Overfitting** | K prilis male - model je prilis citlivy |
| **Underfitting** | K prilis velke - model je prilis jednoduchy |
| **Cross-validation** | Metoda pro nalezeni optimalniho K |

### Kontrolni otazky:

1. **Co se stane, kdyz zvolime K=1?**
2. **Proc je rozhodovaci hranice 'zubata' pro male K?**
3. **Jak poznate overfitting z grafu Train vs Test?**
4. **Proc KNN nepotrebuje 'trenovat'?**

In [ ]:
# Shrnujici vizualizace
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Graf 1: Overfitting (K=1)
vizualizuj_hranici(X, y, k=1, ax=axes[0], title='OVERFITTING (K=1)')

# Graf 2: Optimalni K
vizualizuj_hranici(X, y, k=nejlepsi_k_cv, ax=axes[1], title=f'OPTIMALNI (K={nejlepsi_k_cv})')

# Graf 3: Underfitting
vizualizuj_hranici(X, y, k=30, ax=axes[2], title='UNDERFITTING (K=30)')

plt.suptitle('Porovnani: Overfitting vs Optimalni vs Underfitting', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Test vasich znalosti
print("=" * 60)
print("KVIZ: TEST VASICH ZNALOSTI")
print("=" * 60)

otazky = [
    {
        "otazka": "Co znamena K=3 v KNN?",
        "moznosti": ["A) Model ma 3 vrstvy", 
                     "B) Ptame se 3 nejblizsich sousedu", 
                     "C) Mame 3 tridy"],
        "spravne": "B"
    },
    {
        "otazka": "Co zpusobuje K=1?",
        "moznosti": ["A) Underfitting", 
                     "B) Overfitting", 
                     "C) Optimalni vysledky"],
        "spravne": "B"
    },
    {
        "otazka": "Proc je KNN 'liny ucitel'?",
        "moznosti": ["A) Nepotrebuje zadna data", 
                     "B) Netrénuje se - jen si pamatuje data", 
                     "C) Pracuje velmi pomalu"],
        "spravne": "B"
    }
]

for i, q in enumerate(otazky, 1):
    print(f"\nOtazka {i}: {q['otazka']}")
    for m in q['moznosti']:
        print(f"  {m}")
    print(f"  --> Spravna odpoved: {q['spravne']}")

## 9. Vase vyzva

1. **Zkuste ruzne hodnoty K** - jak se meni rozhodovaci hranice?
2. **Pouzijte jiny dataset** - zkuste Iris se vsemi tremi tridami
3. **Normalizujte data** - co se stane, kdyz maji priznaky ruzne meritko?

In [ ]:
# Vas prostor pro experimentovani
# -----------------------------

# Tip: Zkuste pouzit vsechny tri tridy Iris
# iris = load_iris()
# X_full = iris.data[:, :2]  # Prvni dva priznaky
# y_full = iris.target

# Tip: Normalizace dat
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

print("Experimentujte s kodem vyse!")